## Review Analysis using RNN

Sentiment Analysis of movie reviews using Recurrent Neural Networks and LSTM.

Importing the dataset

In [1]:
import tensorflow as tf
import numpy as np
import time
from collections import Counter

In [2]:
reviews = ''
labels = ''

with open('data/reviews.txt', 'r') as f:
    reviews = f.read()
    
with open('data/labels.txt', 'r') as f:
    labels = f.read()

In [3]:
print ('Sample Positive Review-->')
print (reviews.split('\n')[0])

Sample Positive Review-->
bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   


In [4]:
print ('Sample Negative Review-->')
print (reviews.split('\n')[1])

Sample Negative Review-->
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turned into an insane  violent mob by the crazy chantings of it  s singers . unfortunately it stays absurd the whole time with no general narrative eventually making it just too off putting . even those from the era should be turned off . the cryptic dialogue would make shakespeare seem easy to a third grader . on a technical level it  s better than you might think with some good cinematography by future great vilmos zsigmond . future stars sally kirkland and frederic forrest can be seen briefly .  


In [5]:
len(labels.split('\n'))

25000

## Some data analysis

In [7]:
total_words = len(reviews.split())
total_characters = len(reviews)
unique_words = len(set(reviews.split()))
unique_characters = len(set(reviews))

print ('FOR REVIEWS')
print ("Total words :", total_words)
print ("Total characters :", total_characters)
print ("Unique words :", unique_words)
print ("Unique characters:", unique_characters)

FOR REVIEWS
Total words : 6347388
Total characters : 33678267
Unique words : 74073
Unique characters: 29


In [8]:
from string import punctuation

all_text = ''.join([c for c in reviews if c not in punctuation])
reviews = all_text.split('\n')
labels = labels.split('\n')

all_text = ' '.join(reviews)
words = all_text.split()

In [13]:
len(labels)

25000

In [9]:
positive_counts = Counter()
negative_counts = Counter()
total_counts = Counter()

In [11]:
for i in range(len(reviews)):
    if(labels[i] == 'positive'):
        for word in reviews[i].split():
            positive_counts[word] += 1
            total_counts[word] += 1
    else:
        for word in reviews[i].split():
            negative_counts[word] += 1
            total_counts[word] += 1

IndexError: list index out of range

In [ ]:
positive_counts.most_common()[:20]

In [ ]:
negative_counts.most_common()[:20]

In [ ]:
pos_neg_ratios = Counter()
for term,cnt in list(total_counts.most_common()):
    if(cnt > 100):
        pos_neg_ratio = positive_counts[term] / float(negative_counts[term]+1)
        pos_neg_ratios[term] = pos_neg_ratio

In [ ]:
print("Pos-to-neg ratio for 'the' = {}".format(pos_neg_ratios["the"]))
print("Pos-to-neg ratio for 'amazing' = {}".format(pos_neg_ratios["amazing"]))
print("Pos-to-neg ratio for 'terrible' = {}".format(pos_neg_ratios["terrible"]))

In [ ]:
for word,ratio in pos_neg_ratios.most_common():
    pos_neg_ratios[word] = np.log(ratio)

In [ ]:
print("Pos-to-neg ratio for 'the' = {}".format(pos_neg_ratios["the"]))
print("Pos-to-neg ratio for 'amazing' = {}".format(pos_neg_ratios["amazing"]))
print("Pos-to-neg ratio for 'terrible' = {}".format(pos_neg_ratios["terrible"]))

In [ ]:
pos_neg_ratios.most_common()[:30]

Encoding words to integers. Buidling a dictionary to convert words to integers.

In [14]:
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

reviews_ints = []
for each in reviews:
    reviews_ints.append([vocab_to_int[word] for word in each.split()])

In [17]:
vocab_to_int

{'jc': 16371,
 'sensitivity': 8055,
 'swooning': 28785,
 'lashley': 60296,
 'imagines': 11980,
 'interviewing': 12397,
 'sayonara': 15098,
 'maxx': 19869,
 'kibbutznikim': 73012,
 'gameboys': 46418,
 'catastrophic': 14045,
 'mainstays': 46419,
 'holed': 20645,
 'flaws': 1491,
 'crying': 2540,
 'choronzhon': 46420,
 'arora': 46421,
 'excoriated': 46422,
 'pelting': 46423,
 'bigas': 24103,
 'whats': 4539,
 'spying': 13583,
 'dilapidated': 14557,
 'obsessives': 64482,
 'storage': 13154,
 'dizzyingly': 46425,
 'blade': 4421,
 'viscontian': 55594,
 'lockley': 51688,
 'declining': 16372,
 'terminology': 18839,
 'soundless': 67282,
 'trois': 21025,
 'trellis': 46427,
 'origonal': 46428,
 'tugging': 19870,
 'lever': 18840,
 'silicone': 19871,
 'assurance': 15686,
 'clarrissa': 46429,
 'friendliest': 37443,
 'shamroy': 32300,
 'nra': 26225,
 'republics': 46430,
 'livelihood': 28500,
 'kake': 46431,
 'hanson': 11302,
 'quota': 19872,
 'resulting': 4975,
 'xo': 46433,
 'sentence': 4089,
 'fundame

In [18]:
print (reviews[0])
x = []
for i in reviews[0].split():
    x.append(vocab_to_int[i])
print ('\n')    
print (x)    

bromwell high is a cartoon comedy  it ran at the same time as some other programs about school life  such as  teachers   my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers   the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students  when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled          at           high  a classic line inspector i  m here to sack one of your teachers  student welcome to bromwell high  i expect that many adults of my age think that bromwell high is far fetched  what a pity that it isn  t   


[21141, 308, 6, 3, 1050, 207, 8, 2139, 32, 1, 171, 57, 15, 49, 81, 5817, 44, 382, 110, 140, 15, 5232, 60, 154, 9, 1, 4977, 5866, 475, 71, 5, 260, 12, 21141, 308, 13, 1978, 6, 74, 2401, 5, 613,

Encoding labels to 0(negative) or 1(positive)

In [19]:
labels = np.array([1 if each == 'positive' else 0 for each in labels])

Counting number of zero length reviews

In [20]:
non_zero_idx = [ii for ii, review in enumerate(reviews_ints) if len(review) != 0]
len(non_zero_idx)

25000

In [21]:
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum review length: 2514


In [22]:
reviews_ints = [reviews_ints[ii] for ii in non_zero_idx]
labels = np.array([labels[ii] for ii in non_zero_idx])

In [23]:
len(reviews_ints)

25000

In [24]:
seq_length = 300
features = np.zeros((len(reviews_ints), seq_length), dtype=int)
for i, row in enumerate(np.array(reviews_ints)):
    features[i, -len(row):] = np.array(row)[:seq_length]

In [25]:
features[:2,:]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [26]:
split_frac = 0.8
split_idx = int(len(features)*0.8)
train_x, val_x = features[:split_idx], features[split_idx:]
train_y, val_y = labels[:split_idx], labels[split_idx:]

test_idx = int(len(val_x)*0.5)
val_x, test_x = val_x[:test_idx], val_x[test_idx:]
val_y, test_y = val_y[:test_idx], val_y[test_idx:]

print("Train set:{}".format(train_x.shape), 
      "\nValidation set:{}".format(val_x.shape),
      "\nTest set: {}".format(test_x.shape))

Train set:(20000, 300) 
Validation set:(2500, 300) 
Test set: (2500, 300)


In [27]:
lstm_size = 512
lstm_layers = 2
batch_size = 64
learning_rate = 0.001
n_words = len(vocab_to_int) + 1
embed_size = 300

## Buidling the RNN Model

Defining the placeholder for feeding the input data

In [28]:
def placeholders():
    inputs = tf.placeholder(tf.int32, shape=(None, None), name='inputs')
    targets = tf.placeholder(tf.int32, shape=(None), name='targets')
    keep_prob = tf.placeholder(tf.float32, name= 'keep_prob')
    
    return inputs, targets, keep_prob

Building the embedding lookup matrix to get the embedded vectors to pass to the LSTM cell.

In [29]:
def create_embedding(n_words, embed_size, inputs):
    embedding_matrix = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding_matrix, inputs)
    
    return embedding_matrix, embed

Creating LSTM cells for our RNN

In [30]:
def lstm_cell(lstm_size, lstm_layers, batch_size, keep_prob):
    
    def build_cell(lstm_size, keep_prob):
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(lstm_layers)])
    state = cell.zero_state(batch_size, tf.float32)
    
    return cell, state

Function for returning batches from our data

In [31]:
def generate_batches(x, y, batch_size=100):
    
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

Setting up all variables and placeholders

In [32]:
tf.reset_default_graph()

## Getting input tensors
inputs, targets, keep_prob = placeholders()

embedding_matrix, embed = create_embedding(n_words, embed_size, inputs)

## Creating LSTM Cell
cell, initial_state = lstm_cell(lstm_size, lstm_layers, batch_size, keep_prob)

## Collect outputs(RNN Forward Pass)
outputs, final_state = tf.nn.dynamic_rnn(cell, embed, initial_state = initial_state)

## Predictions
predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn = tf.sigmoid)

## Cost - Mean Squared Error
cost = tf.losses.mean_squared_error(targets, predictions)

## Gradient Descent Step - Backpropagation
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

## Calculating accuracy and correct predictions
correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), targets)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Training the RNN

In [33]:
epochs = 10

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        
        for ii, (x, y) in enumerate(generate_batches(train_x, train_y, batch_size), 1):
            feed = {inputs: x,
                    targets: y,
                    keep_prob: 0.5,
                    initial_state: state}
            loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)
            
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))

            if iteration%25==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in generate_batches(val_x, val_y, batch_size):
                    feed = {inputs: x,
                            targets: y,
                            keep_prob: 1,
                            initial_state: val_state}
                    batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
    saver.save(sess, "checkpoints/sentiment.ckpt")

Epoch: 0/10 Iteration: 5 Train loss: 0.251
Epoch: 0/10 Iteration: 10 Train loss: 0.250
Epoch: 0/10 Iteration: 15 Train loss: 0.251
Epoch: 0/10 Iteration: 20 Train loss: 0.250
Epoch: 0/10 Iteration: 25 Train loss: 0.250
Val acc: 0.500
Epoch: 0/10 Iteration: 30 Train loss: 0.251
Epoch: 0/10 Iteration: 35 Train loss: 0.250
Epoch: 0/10 Iteration: 40 Train loss: 0.250
Epoch: 0/10 Iteration: 45 Train loss: 0.250
Epoch: 0/10 Iteration: 50 Train loss: 0.250
Val acc: 0.500
Epoch: 0/10 Iteration: 55 Train loss: 0.250
Epoch: 0/10 Iteration: 60 Train loss: 0.250
Epoch: 0/10 Iteration: 65 Train loss: 0.250
Epoch: 0/10 Iteration: 70 Train loss: 0.250
Epoch: 0/10 Iteration: 75 Train loss: 0.250
Val acc: 0.500
Epoch: 0/10 Iteration: 80 Train loss: 0.250
Epoch: 0/10 Iteration: 85 Train loss: 0.250
Epoch: 0/10 Iteration: 90 Train loss: 0.250
Epoch: 0/10 Iteration: 95 Train loss: 0.250
Epoch: 0/10 Iteration: 100 Train loss: 0.250
Val acc: 0.500
Epoch: 0/10 Iteration: 105 Train loss: 0.250
Epoch: 0/10 Ite

KeyboardInterrupt: 

Evaluating Accuracy on Test set

In [ ]:
test_acc = []
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {inputs_: x,
                labels_: y[:, None],
                keep_prob: 1,
                initial_state: test_state}
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.3f}".format(np.mean(test_acc)))